# Add Correction filing for All active existing companies

<b> Purpose: Add Corrections filing for all active existing BENs.</b>

This is a one time (python) script to be run at a given date/time.<br>
Set the configuration (client_id, client_secret, url(s)) for a scpecific environment.<br>
Get access token for authorization.


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import psycopg2
import pandas as pd

# this will load all the envars from a .env file located in the project root (api)
load_dotenv(find_dotenv())

%load_ext sql

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('ENTITY_DATABASE_USERNAME', '') + ":" + os.getenv('ENTITY_DATABASE_PASSWORD', '') +'@' + \
                os.getenv('ENTITY_DATABASE_HOST', '') + ':' + os.getenv('ENTITY_DATABASE_PORT', '5432') + '/' + \
                os.getenv('ENTITY_DATABASE_NAME', '');
connect_to_db
    
%sql $connect_to_db

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

In [ ]:
import requests
import os
from datetime import datetime

# token_url, client_id, client_secret, base_url - update based on environment
token_url = os.getenv('ACCOUNT_SVC_AUTH_URL')
client_id = os.getenv('ACCOUNT_SVC_CLIENT_ID')
client_secret = os.getenv('ACCOUNT_SVC_CLIENT_SECRET')
base_url = os.getenv('LEGAL_API_BASE_URL')

header = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = 'grant_type=client_credentials'

res = requests.post(token_url, data, auth=(client_id, client_secret), headers=header)

# Check the status code of the response
if res.status_code == 200:
    print(f"Access token returned successfully : {base_url}")
    token = res.json()["access_token"]
else:
    print(f"Failed to make POST request. Status code: {res.status_code}")
    print(res.text)  # Print the error message if the request fails



Call API (POST) endpoint to createCorrection filing with details as Ben correction statement for businesses.

In [ ]:
from urllib.parse import urljoin
from corrections_output_ia import businesses

current_date_time = datetime.now()
current_date = datetime.now().date().isoformat()
formatted_current_date = datetime.now().date().strftime('%B %d, %Y')
correction_statement = ("BC benefit company statement contained in notice of articles as required under section " 
"51.992 of the Business Corporations Act corrected from “This company is a benefit company and, as such, has purposes "
"that include conducting its business in a responsible and sustainable manner and promoting one or more public "
"benefits” to “This company is a benefit company and, as such, is committed to conducting its business in a "
"responsible and sustainable manner and promoting one or more public benefits”")

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

successful_identifiers = []
failed_identifiers = []
skipped_identifiers = []

# loop through list of businesses to create filing
for identifier in businesses:
    filing_details = %sql \
        SELECT f.id, f.filing_date \
        FROM businesses b \
        JOIN filings f ON b.id = f.business_id \
        WHERE f.filing_type = 'incorporationApplication' \
        AND b.identifier = :identifier
                              
    if filing_details:
        filing_id = filing_details[0]['id']
        filing_date = filing_details[0]['filing_date']

        formatted_filing_date = filing_date.strftime("%B %d, %Y")
    
    draft_details = %sql \
        SELECT b.state, \
            (SELECT COUNT(1) \
                FROM filings f \
                WHERE f.business_id = b.id \
                AND f.status in ('DRAFT', 'PENDING')) <> 0 AS has_draft \
        FROM businesses b \
        WHERE b.identifier = :identifier
    
    state = None
    has_draft = None
    if draft_details:
        state = draft_details[0]['state']
        has_draft = draft_details[0]['has_draft']
    
    if state == 'HISTORICAL' or has_draft:
        skipped_identifiers.append(identifier)
        continue
    
    correction_filing_data = {
        "filing": {
            "header": {
                "name": "correction",
                "date": current_date,
                "certifiedBy": "system",
                "correctionBenStatement": True,
                "waiveFees": True
            },
            "business": {
                "identifier": identifier,
                "legalType": "BEN"
            },
            "correction": {
                "details": "BEN Correction statement",
                "correctedFilingId": filing_id,
                "correctedFilingType": "incorporationApplication",
                "commentOnly": True,
                "comment": f"""Correction for Incorporation Application filed on {formatted_filing_date} \n{correction_statement}"""
            }
        }
    }

    filing_url = urljoin(base_url, f"/api/v2/businesses/{identifier}/filings")
    rv = requests.post(filing_url, headers=headers, json=correction_filing_data)

    # Check the status code of the response
    if rv.status_code == 201:
        successful_identifiers.append(identifier)
    else:
        failed_identifiers.append(identifier)
        print(f"Failed to make POST request. Status code: {rv.status_code}: {rv.text} for {identifier}")
print('Successfully filed Corrections for:', successful_identifiers) # Print the successful identifiers
print('Failed to file Corrections for:', failed_identifiers) # Print the failed identifiers
print('Skipped to file Corrections for:', skipped_identifiers) # Print the skipped identifiers
